Before starting this practice, visit the PLIP site to investigate what PLIP actually analyzes
- https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index

In [1]:
import os
import pandas as pd
import nglview as nv

from Bio.PDB import PDBParser, PDBIO
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.Structure import Structure
from Bio.PDB.Chain import Chain
from Bio.PDB.Model import Model
from Bio.PDB.Atom import Atom
from Bio.PDB.Residue import Residue
from openbabel import pybel

from plip.structure.preparation import PDBComplex

In [2]:
dir_ = '/home/seongok/works/practice/small_molecule_tutorial/2025_KIDDS'

receptor_path = os.path.join(dir_, 'receptor.pdb')
ligand_mol2_path = os.path.join(dir_, 'ligands.mol2')
complex_path = os.path.join(dir_, 'complex.pdb')

# 1. Merge receptor and ligand into a single complex PDB

In [3]:
def prepare_complex(
        dir_,
        ligand_mol2_path,
        receptor_path,
        complex_path,
        pose_idx,
    ):
    prefix, format_ = ligand_mol2_path.split('.')
    m = list(pybel.readfile(format_, ligand_mol2_path))[pose_idx]
    m.addh()

    ligand_pdb = prefix + '.pdb'
    m.write('pdb', ligand_pdb, overwrite=True)

    f_ligand = open(ligand_pdb, 'r')
    ligand_lines = f_ligand.readlines()
    f_ligand.close()

    f_receptor = open(receptor_path, 'r')
    receptor_lines = f_receptor.readlines()
    f_receptor.close()

    f = open(complex_path, 'w')
    for line in receptor_lines:
        if line.startswith('ATOM') or line.startswith('HETATM') or line.startswith('TER'):
            f.write(line)
    f.write('TER\n')
    for line in ligand_lines[2:]:
        if line.startswith('ATOM') or line.startswith('HETATM'):
            f.write(line)
    f.write('END\n')
    f.close()

In [4]:
prepare_complex(
    dir_=dir_,
    ligand_mol2_path=ligand_mol2_path,
    receptor_path=receptor_path,
    complex_path=complex_path,
    pose_idx=0
)

In [5]:
f = open(complex_path, 'r')
lines = f.readlines()
f.close()

# 2. PLIP analysis

In [6]:
mol = PDBComplex()
mol.load_pdb(complex_path)
mol.analyze()

interactions = mol.interaction_sets
interactions

{'MOL:A:1': <plip.structure.preparation.PLInteraction at 0x14837febff40>}

In [7]:
key_list = interactions.keys()
key_list

dict_keys(['MOL:A:1'])

# 3. Extract interactions

In [8]:
def extract_interaction(
        complex_pdb,
        ligand_id='MOL'
    ):
    mol = PDBComplex()
    mol.load_pdb(complex_pdb)
    mol.analyze()

    interactions = mol.interaction_sets
    key_list = interactions.keys()
    for key in key_list:
        if ligand_id in key:
            interaction = interactions[key]

            res_list = []
            for content in interaction.hbonds_ldon:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            for content in interaction.hbonds_pdon:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            res_list = sorted(res_list)
            res_hbonds = ':'.join([str(res) for res in res_list])
            n_hbonds = len(res_list)

            res_list = []
            for content in interaction.pication_laro:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            for content in interaction.pication_paro:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            res_list = sorted(res_list)
            res_pication = ':'.join([str(res) for res in res_list])
            n_pication = len(res_list)

            res_list = []
            for content in interaction.pistacking:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            res_list = sorted(res_list)
            res_pistack = ':'.join([str(res) for res in res_list])
            n_pistack = len(res_list)

            res_list = []
            for content in interaction.hydrophobic_contacts:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            res_list = sorted(res_list)
            res_hydrophobic = ':'.join([str(res) for res in res_list])
            n_hydrophobic = len(res_list)

            res_list = []
            for content in interaction.halogen_bonds:
                res_ = content.resnr
                if res_ not in res_list:
                    res_list.append(res_)
            res_list = sorted(res_list)
            res_halogen = ':'.join([str(res) for res in res_list])
            n_halogen = len(res_list)

            output = [
                n_hbonds,
                n_pication,
                n_pistack,
                n_hydrophobic,
                n_halogen,
                res_hbonds,
                res_pication,
                res_pistack,
                res_hydrophobic,
                res_halogen,
            ]
            return output

In [9]:
extract_interaction(complex_path)

[1, 0, 1, 5, 0, '905', '', '905', '884:890:901:902:905', '']

# 4. Analyze for all ligands

In [10]:
m = list(pybel.readfile('mol2', ligand_mol2_path))
len(m)

16

In [11]:
profile_list = []
for idx in range(len(m)):
    complex_path = os.path.join(dir_, 'complex_'+str(idx)+'.pdb')
    prepare_complex(
        dir_=dir_,
        ligand_mol2_path=ligand_mol2_path,
        receptor_path=receptor_path,
        complex_path=complex_path,
        pose_idx=idx
    )

    profile = extract_interaction(complex_path)
    profile_list.append(profile)
    os.system('rm ' + complex_path) # To remove complex PDB

In [12]:
profile_list[0]

[1, 0, 1, 5, 0, '905', '', '905', '884:890:901:902:905', '']

In [13]:
columns = [
    'Num_HBond',
    'Num_Pication',
    'Num_Pistack',
    'Num_Hydrophobic',
    'Num_Halogen',
    'Res_HBond',
    'Res_Pication',
    'Res_Pistack',
    'Res_Hydrophobic',
    'Res_Halogen',
]
df = pd.DataFrame(profile_list, columns=columns)

In [14]:
df

,Num_HBond,Num_Pication,Num_Pistack,Num_Hydrophobic,Num_Halogen,Res_HBond,Res_Pication,Res_Pistack,Res_Hydrophobic,Res_Halogen
0,1,0,1,5,0,905,,905,884:890:901:902:905,
1,1,0,1,5,0,905,,905,884:890:901:902:905,
2,2,0,1,4,1,878:905,,905,884:890:901:905,902
3,2,0,1,5,1,878:901,,905,884:890:901:902:905,902
4,1,0,1,5,0,905,,905,884:890:901:902:905,
5,2,0,1,5,0,884:905,,905,884:890:901:902:905,
6,2,0,1,4,1,878:905,,905,884:890:901:905,898
7,2,0,1,5,1,878:901,,905,878:884:890:901:905,902
8,1,0,1,3,0,905,,890,884:901:902,
9,1,0,1,3,0,901,,890,884:901:902,
